In [1]:
from builtins import print
import numpy as np
import pandas as pd
import matplotlib
import random


matplotlib.use('agg')
import matplotlib.pyplot as plt

matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = 'Arial'

import os
import operator
from keras.optimizers import adam_v2
import inspect
from tensorflow.python.keras.optimizers import adamax_v2
from tensorflow.keras.optimizers import Adam
#import utils

#from utils.constants import UNIVARIATE_DATASET_NAMES as DATASET_NAMES
#from utils.constants import UNIVARIATE_ARCHIVE_NAMES as ARCHIVE_NAMES

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

import keras
from keras.layers.normalization.layer_normalization import *
from keras.layers.normalization.batch_normalization import *
#from utils.utils import calculate_metrics
#from utils.utils import create_directory
#from utils.utils import check_if_file_exits
import gc
import time

#from utils.utils import save_logs
#from utils.utils import save_test_duration

#from utils.utils import read_all_datasets
#from utils.utils import transform_labels
#from utils.utils import create_directory
#from utils.utils import run_length_xps
#from utils.utils import generate_results_csv
,
import sys
import sklearn

#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#import os
#os.chdir("/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidas/")
#!ls

In [3]:
#open("/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidas/AllCandidas_TRAIN.csv")

In [4]:
def check_if_file_exits(file_name):
    return os.path.exists(file_name)


def readucr(filename, delimiter=','):
    print(filename)
    data = np.loadtxt(filename, delimiter=delimiter)
    Y = data[:, 0]
    X = data[:, 1:]
    return X, Y


def readsits(filename, delimiter=','):
    data = np.loadtxt(filename, delimiter=delimiter)
    Y = data[:, -1]
    X = data[:, :-1]
    return X, Y

In [5]:
def create_directory(directory_path):
    if os.path.exists(directory_path):
        return None
    else:
        try:
            os.makedirs(directory_path)
        except:
            # in case another machine created the path meanwhile !:(
            return None
        return directory_path

In [6]:
def read_dataset(root_dir, archive_name, dataset_name):
    datasets_dict = {}

    file_name = root_dir + '/archives/' + archive_name + '/' + dataset_name + '/' + dataset_name
    x_train, y_train = readucr(file_name + '_TRAIN.csv')
    x_test, y_test = readucr(file_name + '_TEST.csv')
    datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                   y_test.copy())

    return datasets_dict

In [7]:
def read_all_datasets(root_dir, archive_name):
    datasets_dict = {}

    dataset_names_to_sort = []

    if archive_name == 'TSC':
        for dataset_name in UNIVARIATE_DATASET_NAMES:
            root_dir_dataset = root_dir + '\\archives\\' + archive_name + '\\' + dataset_name + '\\'
            file_name = root_dir_dataset + dataset_name
            print(file_name)
            x_train, y_train = readucr(file_name + '_TRAIN.csv')
            x_test, y_test = readucr(file_name + '_TEST.csv')

            datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                           y_test.copy())

            dataset_names_to_sort.append((dataset_name, len(x_train)))

        dataset_names_to_sort.sort(key=operator.itemgetter(1))

        for i in range(len(UNIVARIATE_DATASET_NAMES)):
            UNIVARIATE_DATASET_NAMES[i] = dataset_names_to_sort[i][0]

#    elif archive_name == 'InlineSkateXPs':
#
#        for dataset_name in dataset_names_for_archive[archive_name]:
#            root_dir_dataset = root_dir + '/archives/' + archive_name + '/' + dataset_name + '/'
#
#            x_train = np.load(root_dir_dataset + 'x_train.npy')
#            y_train = np.load(root_dir_dataset + 'y_train.npy')
#            x_test = np.load(root_dir_dataset + 'x_test.npy')
#            y_test = np.load(root_dir_dataset + 'y_test.npy')
#
#            datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
#                                           y_test.copy())
#    elif archive_name == 'SITS':
#        return read_sits_xps(root_dir)
#    else:
#        print('error in archive name')
#        exit()

    return datasets_dict

In [8]:
def calculate_metrics(y_true, y_pred, duration):
    res = pd.DataFrame(data=np.zeros((1, 5), dtype=np.float), index=[0],
                       columns=['precision', 'accuracy', 'recall','f1_score','duration'])
    res['precision'] = precision_score(y_true, y_pred, average='macro')
    res['accuracy'] = accuracy_score(y_true, y_pred)
    res['recall'] = recall_score(y_true, y_pred, average='macro')
    res['f1_score'] = f1_score(y_true, y_pred, average='macro')
    res['duration'] = duration
    return res

In [9]:
def save_test_duration(file_name, test_duration):
    res = pd.DataFrame(data=np.zeros((1, 1), dtype=np.float), index=[0],
                       columns=['test_duration'])
    res['test_duration'] = test_duration
    res.to_csv(file_name, index=False)

In [10]:
def transform_labels(y_train, y_test):
    """
    Transform label to min equal zero and continuous
    For example if we have [1,3,4] --->  [0,1,2]
    """
    # no validation split
    # init the encoder
    encoder = LabelEncoder()
    # concat train and test to fit
    y_train_test = np.concatenate((y_train, y_test), axis=0)
    # fit the encoder
    encoder.fit(y_train_test)
    # transform to min zero and continuous labels
    new_y_train_test = encoder.transform(y_train_test)
    # resplit the train and test
    new_y_train = new_y_train_test[0:len(y_train)]
    new_y_test = new_y_train_test[len(y_train):]
    return new_y_train, new_y_test

In [11]:
def generate_results_csv(output_file_name, root_dir, clfs):
    res = pd.DataFrame(data=np.zeros((0, 8), dtype=np.float), index=[],
                       columns=['classifier_name', 'archive_name', 'dataset_name', 'iteration',
                                'precision', 'accuracy', 'recall', 'duration'])
    for archive_name in UNIVARIATE_ARCHIVE_NAMES:
        datasets_dict = read_all_datasets(root_dir, archive_name)
        for classifier_name in clfs:
            durr = 0.0

            curr_archive_name = archive_name
            for dataset_name in datasets_dict.keys():
                output_dir = root_dir + 'results\\' + classifier_name + '\\' \
                             + curr_archive_name + '\\' + dataset_name + '\\' + 'df_metrics.csv'
                print(output_dir)
                if not os.path.exists(output_dir):
                    continue
                df_metrics = pd.read_csv(output_dir)
                df_metrics['classifier_name'] = classifier_name
                df_metrics['archive_name'] = archive_name
                df_metrics['dataset_name'] = dataset_name
                df_metrics['iteration'] = 0
                res = pd.concat((res, df_metrics), axis=0, sort=False)
                durr += df_metrics['duration'][0]

    res.to_csv(root_dir + output_file_name, index=False)

    res = res.loc[res['classifier_name'].isin(clfs)]

    return res

In [12]:
def plot_epochs_metric(hist, file_name, metric='loss'):
    plt.figure()
    plt.plot(hist.history[metric])
    plt.plot(hist.history['val_' + metric])
    plt.title('model ' + metric)
    plt.ylabel(metric, fontsize='large')
    plt.xlabel('epoch', fontsize='large')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(file_name, bbox_inches='tight')
    plt.close()

In [13]:
def save_logs(output_directory, hist, y_pred, y_true, duration,
              lr=True, plot_test_acc=True):
    hist_df = pd.DataFrame(hist.history)
    hist_df.to_csv(output_directory + 'history.csv', index=False)

    df_metrics = calculate_metrics(y_true, y_pred, duration)
    df_metrics.to_csv(output_directory + 'df_metrics.csv', index=False)

    index_best_model = hist_df['loss'].idxmin()
    row_best_model = hist_df.loc[index_best_model]

    df_best_model = pd.DataFrame(data=np.zeros((1, 6), dtype=np.float), index=[0],
                                 columns=['best_model_train_loss', 'best_model_val_loss', 'best_model_train_acc',
                                          'best_model_val_acc', 'best_model_learning_rate', 'best_model_nb_epoch'])

    df_best_model['best_model_train_loss'] = row_best_model['loss']
    #if plot_test_acc:
    #    df_best_model['best_model_val_loss'] = row_best_model['val_loss']
    #df_best_model['best_model_train_acc'] = row_best_model['acc']
    if plot_test_acc:
        df_best_model['best_model_val_acc'] = row_best_model['val_acc']
    if lr == True:
        df_best_model['best_model_learning_rate'] = row_best_model['lr']
    df_best_model['best_model_nb_epoch'] = index_best_model

    df_best_model.to_csv(output_directory + 'df_best_model.csv', index=False)

    if plot_test_acc:
        # plot losses
        plot_epochs_metric(hist, output_directory + 'epochs_loss.png')

    return df_metrics

In [14]:
def create_synthetic_dataset(pattern_len=[0.25], pattern_pos=[0.1, 0.65], ts_len=128, ts_n=128):
    random.seed(1234)
    np.random.seed(1234)

    nb_classes = len(pattern_pos) * len(pattern_len)

    out_dir = '/b/home/uha/hfawaz-datas/dl-tsc/archives/UCRArchive_2018/BinaryData/'

    create_directory(out_dir)

    x_train = np.random.normal(0.0, 0.1, size=(ts_n, ts_len))
    x_test = np.random.normal(0.0, 0.1, size=(ts_n, ts_len))

    y_train = np.random.randint(low=0, high=nb_classes, size=(ts_n,))
    y_test = np.random.randint(low=0, high=nb_classes, size=(ts_n,))

    # make sure at least each class has one example
    y_train[:nb_classes] = np.arange(start=0, stop=nb_classes, dtype=np.int32)
    y_test[:nb_classes] = np.arange(start=0, stop=nb_classes, dtype=np.int32)

    # each class is defined with a certain combination of pattern_pos and pattern_len
    # with one pattern_len and two pattern_pos we can create only two classes
    # example:  class 0 _____-_  & class 1 _-_____

    # create the class definitions
    class_def = [None for i in range(nb_classes)]

    idx_class = 0
    for pl in pattern_len:
        for pp in pattern_pos:
            class_def[idx_class] = {'pattern_len': int(pl * ts_len),
                                    'pattern_pos': int(pp * ts_len)}
            idx_class += 1

    # create the dataset
    for i in range(ts_n):
        # for the train
        c = y_train[i]
        curr_pattern_pos = class_def[c]['pattern_pos']
        curr_pattern_len = class_def[c]['pattern_len']
        x_train[i][curr_pattern_pos:curr_pattern_pos + curr_pattern_len] = \
            x_train[i][curr_pattern_pos:curr_pattern_pos + curr_pattern_len] + 1.0

        # for the test
        c = y_test[i]
        curr_pattern_pos = class_def[c]['pattern_pos']
        curr_pattern_len = class_def[c]['pattern_len']
        x_test[i][curr_pattern_pos:curr_pattern_pos + curr_pattern_len] = \
            x_test[i][curr_pattern_pos:curr_pattern_pos + curr_pattern_len] + 1.0

    # znorm
    x_train = (x_train - x_train.mean(axis=1, keepdims=True)) \
              / x_train.std(axis=1, keepdims=True)

    x_test = (x_test - x_test.mean(axis=1, keepdims=True)) \
             / x_test.std(axis=1, keepdims=True)

    # visualize example
    # plt.figure()
    # colors = generate_array_of_colors(nb_classes)
    # for c in range(nb_classes):
    #     plt.plot(x_train[y_train == c][0], color=colors[c], label='class-' + str(c))
    # plt.legend(loc='best')
    # plt.savefig('out.pdf')
    # exit()

    # np.save(out_dir+'x_train.npy',x_train)
    # np.save(out_dir+'y_train.npy',y_train)
    # np.save(out_dir+'x_test.npy',x_test)
    # np.save(out_dir+'y_test.npy',y_test)

    # print('Done creating dataset!')

    return x_train, y_train, x_test, y_test

In [15]:
def generate_array_of_colors(n):
    # https://www.quora.com/How-do-I-generate-n-visually-distinct-RGB-colours-in-Python
    ret = []
    r = int(random.random() * 256)
    g = int(random.random() * 256)
    b = int(random.random() * 256)
    alpha = 1.0
    step = 256 / n
    for i in range(n):
        r += step
        g += step
        b += step
        r = int(r) % 256
        g = int(g) % 256
        b = int(b) % 256
        ret.append((r / 255, g / 255, b / 255, alpha))
    return ret

In [16]:
def read_sits_xps(root_dir):
    datasets_dict = {}
    path_to_data = root_dir + 'archives\\SITS\\resampled-SITS\\'
    path_to_test = root_dir + 'archives\\SITS\\' + 'SatelliteFull_TEST_1000.csv'

    x_test, y_test = readsits(path_to_test)

    for subdir, dirs, files in os.walk(path_to_data):
        for file_name in files:
            arr = file_name.split('.')
            dataset_name = arr[0]
            file_type = arr[1]
            if file_type == 'csv':
                x_train, y_train = readsits(subdir + '/' + file_name)

                datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                               y_test.copy())

    return datasets_dict

In [17]:
def resample_dataset(x, rate):
    new_x = np.zeros(shape=(x.shape[0], rate))
    from scipy import signal
    for i in range(x.shape[0]):
        f = signal.resample(x[0], rate)
        new_x[i] = f
    return new_x

In [18]:
def run_length_xps(root_dir):
    #archive_name = ARCHIVE_NAMES[0]
    archive_name = UNIVARIATE_ARCHIVE_NAMES[0]
    dataset_name = 'InlineSkate'
    datasets_dict = read_dataset(root_dir, archive_name, dataset_name)

    lengths = [2 ** i for i in range(5, 12)]

    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]
    x_test = datasets_dict[dataset_name][2]
    y_test = datasets_dict[dataset_name][3]

    new_archive_name = 'TSC'
#    new_archive_name = 'InlineSkateXPs'

    for l in lengths:
        new_x_train = resample_dataset(x_train, l)
        new_x_test = resample_dataset(x_test, l)
        new_dataset_name = dataset_name + '-' + str(l)
        new_dataset_dir = root_dir + 'archives\\' + new_archive_name + '\\' + new_dataset_name + '\\'
        create_directory(new_dataset_dir)

        np.save(new_dataset_dir + 'x_train.npy', new_x_train)
        np.save(new_dataset_dir + 'y_train.npy', y_train)
        np.save(new_dataset_dir + 'x_test.npy', new_x_test)
        np.save(new_dataset_dir + 'y_test.npy', y_test)

In [19]:
#UNIVARIATE_DATASET_NAMES = ['50words', 'Adiac', 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'Car', 'CBF',
#                            'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee',
#                            'Computers', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 'DiatomSizeReduction',
#                            'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect', 'DistalPhalanxTW',
#                            'Earthquakes', 'ECG200', 'ECG5000', 'ECGFiveDays', 'ElectricDevices', 'FaceAll', 'FaceFour',
#                            'FacesUCR', 'FISH', 'FordA', 'FordB', 'Gun_Point', 'Ham', 'HandOutlines',
#                            'Haptics', 'Herring', 'InlineSkate', 'InsectWingbeatSound', 'ItalyPowerDemand',
#                            'LargeKitchenAppliances', 'Lighting2', 'Lighting7', 'MALLAT', 'Meat', 'MedicalImages',
#                            'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxOutlineCorrect', 'MiddlePhalanxTW',
#                            'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 'NonInvasiveFatalECG_Thorax2', 'OliveOil',
#                            'OSULeaf', 'PhalangesOutlinesCorrect', 'Phoneme', 'Plane', 'ProximalPhalanxOutlineAgeGroup',
#                            'ProximalPhalanxOutlineCorrect', 'ProximalPhalanxTW', 'RefrigerationDevices',
#                            'ScreenType', 'ShapeletSim', 'ShapesAll', 'SmallKitchenAppliances', 'SonyAIBORobotSurface',
#                            'SonyAIBORobotSurfaceII', 'StarLightCurves', 'Strawberry', 'SwedishLeaf', 'Symbols',
#                            'synthetic_control', 'ToeSegmentation1', 'ToeSegmentation2', 'Trace', 'TwoLeadECG',
#                            'Two_Patterns', 'UWaveGestureLibraryAll', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y',
#                            'uWaveGestureLibrary_Z', 'wafer', 'Wine', 'WordsSynonyms', 'Worms', 'WormsTwoClass', 'yoga']

#UNIVARIATE_DATASET_NAMES = ['ElectricDevices', 'FordA', 'NonInvasiveFatalECG_Thorax1', 'NonInvasiveFatalECG_Thorax2',
#                            'ProximalPhalanxOutlineCorrect', 'Two_Patterns', 'wafer']

UNIVARIATE_DATASET_NAMES = ['AllCandidas']

#UNIVARIATE_DATASET_NAMES = ['bases_10062021_anem_sens']

#UNIVARIATE_DATASET_NAMES = ['bases_10062021_det', 'bases_10062021']

#UNIVARIATE_DATASET_NAMES = ['nariz_nov2020']

#UNIVARIATE_DATASET_NAMES = ['renan_junto']

#UNIVARIATE_DATASET_NAMES = ['nariz_nov2020', 'renan_junto', 'renan_nao_padrao', 'renan_padrao']

#UNIVARIATE_DATASET_NAMES = ['nariz_1120_9cl_10', 'nariz_1120_9cl_20', 'nariz_1120_9cl_30', 'nariz_1120_9cl_40',
#                            'nariz_1120_9cl_60', 'nariz_1120_9cl_70', 'nariz_1120_9cl_80', 'nariz_1120_9cl_90']

#UNIVARIATE_DATASET_NAMES = ['nariz_nov2020_sem_cl5']

#UNIVARIATE_DATASET_NAMES = ['PhalangesOutlinesCorrect']
#UNIVARIATE_DATASET_NAMES = ['Coffee']

#UNIVARIATE_DATASET_NAMES = ['Meat', 'Coffee']

UNIVARIATE_ARCHIVE_NAMES = ['TSC', 'InlineSkateXPs', 'SITS']
#UNIVARIATE_ARCHIVE_NAMES = ['TSC']

SITS_DATASETS = ['SatelliteFull_TRAIN_c301', 'SatelliteFull_TRAIN_c200', 'SatelliteFull_TRAIN_c451',
                 'SatelliteFull_TRAIN_c89', 'SatelliteFull_TRAIN_c677', 'SatelliteFull_TRAIN_c59',
                 'SatelliteFull_TRAIN_c133']

InlineSkateXPs_DATASETS = ['InlineSkate-32', 'InlineSkate-64', 'InlineSkate-128',
                           'InlineSkate-256', 'InlineSkate-512', 'InlineSkate-1024',
                           'InlineSkate-2048']

dataset_names_for_archive = {'TSC': UNIVARIATE_DATASET_NAMES,
                             'SITS': SITS_DATASETS,
                             'InlineSkateXPs': InlineSkateXPs_DATASETS}

In [20]:
class Classifier_NNE:

    def create_classifier(self, model_name, input_shape, nb_classes, output_directory, verbose=False,
                          build=True):
        if self.check_if_match('inception*', model_name):
            #from classifiers import inception
            return Classifier_INCEPTION(output_directory, input_shape, nb_classes, verbose,build=build)

    def check_if_match(self, rex, name2):
        import re
        pattern = re.compile(rex)
        return pattern.match(name2)

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False, nb_iterations=5,
                 clf_name='inception'):
        self.classifiers = [clf_name]
        out_add = ''
        for cc in self.classifiers:
            out_add = out_add + cc + '-'
        self.archive_name = UNIVARIATE_ARCHIVE_NAMES[0]
        self.iterations_to_take = [i for i in range(nb_iterations)]
        for cc in self.iterations_to_take:
            out_add = out_add + str(cc) + '-'
        self.output_directory = output_directory.replace('nne',
                                                         'nne' + '\\' + out_add)
        create_directory(self.output_directory)
        self.dataset_name = output_directory.split('\\')[-2]
        self.verbose = verbose
        self.models_dir = output_directory.replace('nne', 'classifier')

    def fit(self, x_train, y_train, x_test, y_test, y_true):
        # no training since models are pre-trained
        start_time = time.time()

        y_pred = np.zeros(shape=y_test.shape)

        ll = 0

        # loop through all classifiers
        for model_name in self.classifiers:
            # loop through different initialization of classifiers
            for itr in self.iterations_to_take:
                if itr == 0:
                    itr_str = ''
                else:
                    itr_str = '_itr_' + str(itr)

                curr_archive_name = self.archive_name + itr_str

                curr_dir = self.models_dir.replace('classifier', model_name).replace(
                    self.archive_name, curr_archive_name)

                model = self.create_classifier(model_name, None, None,
                                               curr_dir, build=False)

                predictions_file_name = curr_dir + 'y_pred.npy'
                # check if predictions already made
                if check_if_file_exits(predictions_file_name):
                    # then load only the predictions from the file
                    curr_y_pred = np.load(predictions_file_name)
                else:
                    # then compute the predictions
                    curr_y_pred = model.predict(x_test, y_true, x_train, y_train, y_test,
                                                return_df_metrics=False)
                    keras.backend.clear_session()

                    np.save(predictions_file_name, curr_y_pred)

                y_pred = y_pred + curr_y_pred

                ll += 1

        # average predictions
        y_pred = y_pred / ll

        # save predictions
        np.save(self.output_directory + 'y_pred.npy', y_pred)

        # convert the predicted from binary to integer
        y_pred = np.argmax(y_pred, axis=1)

        duration = time.time() - start_time

        df_metrics = calculate_metrics(y_true, y_pred, duration)

        df_metrics.to_csv(self.output_directory + 'df_metrics.csv', index=False)

        gc.collect()

In [21]:
class Classifier_INCEPTION:

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False, build=True, batch_size=64,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41, nb_epochs=1500):

        self.output_directory = output_directory

        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.batch_size = batch_size
        self.bottleneck_size = 32
        self.nb_epochs = nb_epochs

        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
            self.model.save_weights(self.output_directory + 'model_init.hdf5')

    def _inception_module(self, input_tensor, stride=1, activation='linear'):

        if self.use_bottleneck and int(input_tensor.shape[-1]) > 1:
            input_inception = keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = keras.layers.Concatenate(axis=2)(conv_list)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation(activation='relu')(x)
        return x

    def _shortcut_layer(self, input_tensor, out_tensor):
        shortcut_y = keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                         padding='same', use_bias=False)(input_tensor)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        x = keras.layers.Add()([shortcut_y, out_tensor])
        x = keras.layers.Activation('relu')(x)
        return x

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        x = input_layer
        input_res = input_layer

        for d in range(self.depth):

            x = self._inception_module(x)

            if self.use_residual and d % 3 == 2:
                x = self._shortcut_layer(input_res, x)
                input_res = x

        gap_layer = keras.layers.GlobalAveragePooling1D()(x)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)
        lr = 00.5
        optimizer = adam_v2.Adam(learning_rate=lr, decay=lr/self.nb_epochs)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer,
                      metrics=['accuracy'])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50,
                                                      min_lr=0.0001)

        file_path = self.output_directory + 'best_model.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                           save_best_only=True)

        self.callbacks = [reduce_lr, model_checkpoint]

        return model

    def fit(self, x_train, y_train, x_val, y_val, y_true, plot_test_acc=False):
        #if len(keras.backend.tensorflow_backend._get_available_gpus()) == 0:       
        #    print('error no gpu')
        #    exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training

        if self.batch_size is None:
            mini_batch_size = int(min(x_train.shape[0] / 10, 16))
        else:
            mini_batch_size = self.batch_size

        start_time = time.time()

        if plot_test_acc:

            hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=self.nb_epochs,
                                  verbose=self.verbose, validation_data=(x_val, y_val), callbacks=self.callbacks)
        else:

            hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=self.nb_epochs,
                                  verbose=self.verbose, callbacks=self.callbacks)

        duration = time.time() - start_time

        self.model.save(self.output_directory + 'last_model.hdf5')

        y_pred = self.predict(x_val, y_true, x_train, y_train, y_val,
                              return_df_metrics=False)

        # save predictions
        np.save(self.output_directory + 'y_pred.npy', y_pred)

        # convert the predicted from binary to integer
        y_pred = np.argmax(y_pred, axis=1)

        df_metrics = save_logs(self.output_directory, hist, y_pred, y_true, duration,
                               plot_test_acc=plot_test_acc)

        keras.backend.clear_session()

        return df_metrics

    def predict(self, x_test, y_true, x_train, y_train, y_test, return_df_metrics=True):
        start_time = time.time()
        model_path = self.output_directory + 'best_model.hdf5'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test, batch_size=self.batch_size)
        if return_df_metrics:
            y_pred = np.argmax(y_pred, axis=1)
            df_metrics = calculate_metrics(y_true, y_pred, 0.0)
            return df_metrics
        else:
            test_duration = time.time() - start_time
            save_test_duration(self.output_directory + 'test_duration.csv', test_duration)
            return y_pred

In [22]:
def prepare_data():
    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]
    x_test = datasets_dict[dataset_name][2]
    y_test = datasets_dict[dataset_name][3]

    nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))

    # make the min to zero of labels
    y_train, y_test = transform_labels(y_train, y_test)

    # save orignal y because later we will use binary
    y_true = y_test.astype(np.int64)
    y_true_train = y_train.astype(np.int64)
    # transform the labels from integers to one hot vectors
    enc = sklearn.preprocessing.OneHotEncoder()
    enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
    y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
    y_test = enc.transform(y_test.reshape(-1, 1)).toarray()

    if len(x_train.shape) == 2:  # if univariate
        # add a dimension to make it multivariate with one dimension
        x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

    return x_train, y_train, x_test, y_test, y_true, nb_classes, y_true_train, enc

In [23]:
def fit_classifier():
    input_shape = x_train.shape[1:]
    print("Fiting classifier..")
    print(f"classifier_name: {classifier_name}")
    print(f"input_shape: {input_shape}")
    print(f"nb_classes: {nb_classes}")
    print(f"output_directory: {output_directory}")
    
    classifier = create_classifier(classifier_name, input_shape, nb_classes,
                                   output_directory)

    classifier.fit(x_train, y_train, x_test, y_test, y_true)

In [24]:
def create_classifier(classifier_name, input_shape, nb_classes, output_directory,
                      verbose=False, build=True):
    if classifier_name == 'nne':
        #from classifiers import nne
        return Classifier_NNE(output_directory, input_shape, nb_classes, verbose)

    if classifier_name == 'inception':
        #from classifiers import inception
        return Classifier_INCEPTION(output_directory, input_shape, nb_classes,
                                    verbose, build=build)

In [25]:
def get_xp_val(xp):
    if xp == 'batch_size':
        xp_arr = [16, 32, 128]
    elif xp == 'use_bottleneck':
        xp_arr = [False]
    elif xp == 'use_residual':
        xp_arr = [False]
    elif xp == 'nb_filters':
        xp_arr = [16, 64]
    elif xp == 'depth':
        xp_arr = [3, 9]
    elif xp == 'kernel_size':
        xp_arr = [8, 64]
    else:
        raise Exception('wrong argument')
    return xp_arr

In [26]:
############################################### main

#root_dir = '/b/home/uha/hfawaz-datas/temp-dl-tsc/'

root_dir = "C:\\Users\\mlb\\Teste_VOCs\\3_ModelsIMplementations\\"

xps = ['use_bottleneck', 'use_residual', 'nb_filters', 'depth',
       'kernel_size', 'batch_size']

In [27]:
# run nb_iter_ iterations of Inception on the whole TSC archive
classifier_name = 'inception'
#archive_name = ARCHIVE_NAMES[0]
archive_name = UNIVARIATE_ARCHIVE_NAMES[0]
nb_iter_ = 10

datasets_dict = read_all_datasets(root_dir, archive_name)

for iter in range(nb_iter_):
    print('\t\titer', iter)

    trr = ''
    if iter != 0:
        trr = '_itr_' + str(iter)

    tmp_output_directory = root_dir + 'results\\' + classifier_name + '\\' + archive_name + trr + '\\'

    for dataset_name in dataset_names_for_archive[archive_name]:
        print('\t\t\tdataset_name: ', dataset_name)

        x_train, y_train, x_test, y_test, y_true, nb_classes, y_true_train, enc = prepare_data()

        output_directory = tmp_output_directory + dataset_name + '\\'

        temp_output_directory = create_directory(output_directory)

        if temp_output_directory is None:
            print('Already_done', tmp_output_directory, dataset_name)
            continue

        fit_classifier()

        print('\t\t\t\tDONE')

        # the creation of this directory means
        create_directory(output_directory + '\\DONE')

# run the ensembling of these iterations of Inception
classifier_name = 'inception'

datasets_dict = read_all_datasets(root_dir, archive_name)

tmp_output_directory = root_dir + 'results\\' + classifier_name + '\\' + archive_name + '\\'

for dataset_name in dataset_names_for_archive[archive_name]:
    print('\t\t\tdataset_name: ', dataset_name)

    x_train, y_train, x_test, y_test, y_true, nb_classes, y_true_train, enc = prepare_data()

    output_directory = tmp_output_directory + dataset_name + '\\'

    fit_classifier()

    print('\t\t\t\tDONE')

C:\Users\mlb\Teste_VOCs\3_ModelsIMplementations\\archives\TSC\AllCandidas\AllCandidas
C:\Users\mlb\Teste_VOCs\3_ModelsIMplementations\\archives\TSC\AllCandidas\AllCandidas_TRAIN.csv
C:\Users\mlb\Teste_VOCs\3_ModelsIMplementations\\archives\TSC\AllCandidas\AllCandidas_TEST.csv
		iter 0
			dataset_name:  AllCandidas
Already_done C:\Users\mlb\Teste_VOCs\3_ModelsIMplementations\results\inception\TSC\ AllCandidas
		iter 1
			dataset_name:  AllCandidas
Already_done C:\Users\mlb\Teste_VOCs\3_ModelsIMplementations\results\inception\TSC_itr_1\ AllCandidas
		iter 2
			dataset_name:  AllCandidas
Already_done C:\Users\mlb\Teste_VOCs\3_ModelsIMplementations\results\inception\TSC_itr_2\ AllCandidas
		iter 3
			dataset_name:  AllCandidas
Already_done C:\Users\mlb\Teste_VOCs\3_ModelsIMplementations\results\inception\TSC_itr_3\ AllCandidas
		iter 4
			dataset_name:  AllCandidas
Already_done C:\Users\mlb\Teste_VOCs\3_ModelsIMplementations\results\inception\TSC_itr_4\ AllCandidas
		iter 5
			dataset_name

D:\Pessoais\AnacondaFiles\envs\teste_with_gpu\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
C:\Users\mlb\AppData\Local\Temp/ipykernel_3132/4161292546.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  res = pd.DataFrame(data=np.zeros((1, 1), dtype=np.float), index=[0],
C:\Users\mlb\AppData\Local\Temp/ipykernel_3132/3778290522.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence

				DONE


In [28]:
## this part is for running inception with the different hyperparameters
## listed in the paper, on the whole TSC archive
#
#archive_name = 'TSC'
#classifier_name = 'inception'
#max_iterations = 5
#
#datasets_dict = read_all_datasets(root_dir, archive_name)
#
#for xp in xps:
#
#    xp_arr = get_xp_val(xp)
#
#    print('xp', xp)
#
#    for xp_val in xp_arr:
#        print('\txp_val', xp_val)
#
#        kwargs = {xp: xp_val}
#
#        for iter in range(max_iterations):
#
#            trr = ''
#            if iter != 0:
#                trr = '_itr_' + str(iter)
#            print('\t\titer', iter)
#
#            for dataset_name in dataset_names_for_archive[archive_name]:
#
#                output_directory = root_dir + '/results/' + classifier_name + '/' + '/' + xp + '/' + '/' + str(
#                    xp_val) + '/' + archive_name + trr + '/' + dataset_name + '/'
#
#                print('\t\t\tdataset_name', dataset_name)
#                x_train, y_train, x_test, y_test, y_true, nb_classes, y_true_train, enc = prepare_data()
#
#                # check if data is too big for this gpu
#                size_data = x_train.shape[0] * x_train.shape[1]
#
#                temp_output_directory = create_directory(output_directory)
#
#                if temp_output_directory is None:
#                    print('\t\t\t\t', 'Already_done')
#                    continue
#
#                input_shape = x_train.shape[1:]
#
#                #from classifiers import inception
#
#                classifier = Classifier_INCEPTION(output_directory, input_shape, nb_classes,
#                                                  verbose=False, build=True, **kwargs)
#
#                classifier.fit(x_train, y_train, x_test, y_test, y_true)
#
#                # the creation of this directory means
#                create_directory(output_directory + '/DONE')
#
#                print('\t\t\t\t', 'DONE')
#
## we now need to ensemble each iteration of inception (aka InceptionTime)
##archive_name = ARCHIVE_NAMES[0]
#archive_name = UNIVARIATE_ARCHIVE_NAMES[0]
#classifier_name = 'nne'
#
#datasets_dict = read_all_datasets(root_dir, archive_name)
#
#tmp_output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + '/'
#
#for xp in xps:
#    xp_arr = get_xp_val(xp)
#    for xp_val in xp_arr:
#
#        clf_name = 'inception/' + xp + '/' + str(xp_val)
#
#        for dataset_name in dataset_names_for_archive[archive_name]:
#            x_train, y_train, x_test, y_test, y_true, nb_classes, y_true_train, enc = prepare_data()
#
#            output_directory = tmp_output_directory + dataset_name + '/'
#
#            #from classifiers import nne
#
#            classifier = Classifier_NNE(output_directory, x_train.shape[1:],
#                                            nb_classes, clf_name=clf_name)
#
#            classifier.fit(x_train, y_train, x_test, y_test, y_true)

In [29]:
## this is to generate the archive for the length experiments
#run_length_xps(root_dir)
#
#clfs = []
#itr = '-0-1-2-3-4-'
#inceptionTime = 'nne/inception'
## add InceptionTime: an ensemble of 5 Inception networks
#clfs.append(inceptionTime + itr)
## add InceptionTime for each hyperparameter study
#for xp in xps:
#    xp_arr = get_xp_val(xp)
#    for xp_val in xp_arr:
#        clfs.append(inceptionTime + '/' + xp + '/' + str(xp_val) + itr)
#df = generate_results_csv('results.csv', root_dir, clfs)
#print(df)
#
#
#df_clfs = pd.DataFrame(clfs)
#arq = 'df_clfs.csv'
#filepath = root_dir + '/' + arq
#df_clfs.to_csv(filepath)
#
#arq = 'df.csv'
#filepath = root_dir + '/' + arq
#df.to_csv(filepath)


In [30]:

def gen_results_csv(clf_name, path, root_dir):

    ARCHIVE_NAMES = ['TSC', 'TSC_itr_1', 'TSC_itr_2', 'TSC_itr_3', 'TSC_itr_4', 'TSC_itr_5',
                     'TSC_itr_6', 'TSC_itr_7', 'TSC_itr_8', 'TSC_itr_9'] 
    
    DATASET_NAME = ['anem_col_1006e20072021'] 
   
    res1 = np.zeros((10, 3))
    
    res1 = pd.DataFrame(res1)
    
    res1 = res1.astype(object)
    
    nomes = ['classifier_name', 'archive_name', 'dataset_name']
    res1.columns=nomes
    
    res2 = np.zeros((10, 7))
    
    i = 0
    j = 0
    for archive_name in ARCHIVE_NAMES:
    
        o = 0
        for dataset_name in DATASET_NAME:
    
            output_dir = root_dir + path + archive_name + '\\' \
                         + dataset_name + '\\' + 'df_metrics.csv'
            #print(output_dir)
            #print('j = ',j,'i = ',i)
            if not os.path.exists(output_dir):
                continue
            df_metrics = pd.read_csv(output_dir)
            res1['classifier_name'][j] = clf_name
            res1['archive_name'][j] = archive_name
            res1['dataset_name'][j] = dataset_name
            res2[j,0] = j
            res2[j,1] = o
            res2[j,2] = i
            res2[j,3] = df_metrics['precision']
            res2[j,4] = df_metrics['accuracy']
            res2[j,5] = df_metrics['recall']
            res2[j,6] = df_metrics['duration']
            
            o += 1
            j += 1
        i += 1
    
    
    res2 = pd.DataFrame(res2)
    nomes = ['ord1', 'ord2', 'iteration', 'precision', 'accuracy', 'recall', 'duration']
    res2.columns=nomes
    
    
    res = pd.concat([res1, res2], ignore_index=True, axis=1)
    nomes = ['classifier_name', 'archive_name', 'dataset_name','ord1', 'ord2',
             'iteration', 'precision', 'accuracy', 'recall', 'duration']
    res.columns=nomes
    
    res = res.sort_values(by=['ord2', 'ord1'])
    
    return res

root_dir = "C:\\Users\\mlb\\Teste_VOCs\\3_ModelsIMplementations\\results"

clf_name = 'candidas_inception_result'

path = '\\'

res = gen_results_csv(clf_name, path, root_dir)

print(res)

arq = clf_name + '.csv'
filepath = root_dir + path + arq
res.to_csv(filepath, index=False)


  classifier_name archive_name dataset_name  ord1  ord2  iteration  precision  \
0             0.0          0.0          0.0   0.0   0.0        0.0        0.0   
1             0.0          0.0          0.0   0.0   0.0        0.0        0.0   
2             0.0          0.0          0.0   0.0   0.0        0.0        0.0   
3             0.0          0.0          0.0   0.0   0.0        0.0        0.0   
4             0.0          0.0          0.0   0.0   0.0        0.0        0.0   
5             0.0          0.0          0.0   0.0   0.0        0.0        0.0   
6             0.0          0.0          0.0   0.0   0.0        0.0        0.0   
7             0.0          0.0          0.0   0.0   0.0        0.0        0.0   
8             0.0          0.0          0.0   0.0   0.0        0.0        0.0   
9             0.0          0.0          0.0   0.0   0.0        0.0        0.0   

   accuracy  recall  duration  
0       0.0     0.0       0.0  
1       0.0     0.0       0.0  
2       0.0 